In [1]:
#importing useful libraries
import numpy as np
import pandas as pd
import cv2
import math

In [141]:
# arr = np.array([[ 6, 11, 10,  7],[14,  3, 15,  2],[ 9,  8, 13,  4],[ 1, 16,  5, 12]])
arr = cv2.imread('correct_scr.png')
arr = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
print(arr)
m,n = arr.shape
print(m,n)
r = math.ceil(min(m,n)/2)
print(r)
# cv2.imshow("image", arr)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[126  72 127 ...  63  53  74]
 [ 70 145  79 ...  42 122  55]
 [ 69 110  62 ... 100  75 136]
 ...
 [ 13  12  29 ...  39  21  34]
 [ 50  91  53 ...   9  17   9]
 [ 13  73  15 ...  98  70 113]]
400 500
200


In [142]:
#rotate 90 degree clockwise
arr = np.rot90(arr, k = -1)
print(arr)

[[ 13  50  13 ...  69  70 126]
 [ 73  91  12 ... 110 145  72]
 [ 15  53  29 ...  62  79 127]
 ...
 [ 98   9  39 ... 100  42  63]
 [ 70  17  21 ...  75 122  53]
 [113   9  34 ... 136  55  74]]


In [143]:
img_1 = arr.reshape(-1,order = 'F')
img_1

array([13, 73, 15, ..., 63, 53, 74], dtype=uint8)

In [144]:
#extract subarrays from the one_d array. (vector segmentation)
img = []
start = 0
for i in range(1,r+1):
    l = (2*(m+n)+4)- 8*i
    subarray = img_1[start:start+l]
    img.append(subarray)
    start = start+l
print(img[0])

[13 73 15 ...  9 83 10]


In [145]:
#arrange the matrices in the one after one form
img_2 = []
for odarr in img:
    shuffled_1 = odarr[::2]
    shuffled_2 = odarr[1::2]
    shuffled = np.concatenate((shuffled_1,shuffled_2))
    img_2.append(shuffled)
print(img_2[0])

[13 15 14 ... 16  9 10]


In [146]:
#make (r-1) hollow matrices in which these subarrrays gets fit (vector reconstruction)
img_3 = []
for i in range(r):
    mat = np.zeros((m-2*i,n-2*i),np.uint8)
    mat[:,0] = img_2[i][:m-2*i]
    mat[m-2*i-1,1:n-2*i-1] = img_2[i][m-2*i:m+n-4*i-2]
    mat[m-2*i-1::-1,n-2*i-1] = img_2[i][m+n-4*i-2:2*m+n-6*i-2]
    mat[0,n-2*i-2:0:-1] = img_2[i][2*m+n-6*i-2:]
    img_3.append(mat)
print(img_3[1])

[[ 19  13  11 ...  27  17  10]
 [ 18   0   0 ...   0   0  12]
 [  4   0   0 ...   0   0  10]
 ...
 [  4   0   0 ...   0   0 103]
 [ 12   0   0 ...   0   0  73]
 [  2  18  33 ...  79  77  89]]


In [147]:
# i want to insert matrix in to the hollow matrix such that all r matrix goes in the matrix one by one
for i in range(r-1):
    img_3[0][i+1:m-i-1,i+1] = img_3[i+1][:,0]
    img_3[0][m-i-2,i+1:n-i-1] = img_3[i+1][m-2*(i+1)-1,:]
    img_3[0][i+1:m-i-1,n-i-2]  = img_3[i+1][:,n-2*(i+1)-1]
    img_3[0][i+1,i+1:n-i-1] = img_3[i+1][0,:]
print(img_3[0])
# cv2.imwrite("correct_unscr.png",img_3[0])

[[ 13  10   9 ...  16   8  15]
 [ 15  19  13 ...  17  10  13]
 [ 14  18  10 ...   9  12  16]
 ...
 [ 55  12  23 ...  77  73  92]
 [ 57   2  18 ...  77  89  86]
 [ 44   8  21 ... 103  83  73]]


True

In [148]:
cv2.imshow("image",img_3[0])
cv2.waitKey(0)
cv2.destroyAllWindows()